# Specialized Task Shallow Detectors

*   Using latest email from Baker



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
drivepath = '/content/gdrive/MyDrive/Spring_2021/11785_Intro_to_Deep_Learning/DL_Group_Project'
datapath = '/content/gdrive/MyDrive/Spring_2021/11785_Intro_to_Deep_Learning/dl-group-project-code/preprocessed_data'
output_path = f'{drivepath}/experiments/specialized_detectors/'

In [10]:
!pip install tqdm

In [94]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os
from tqdm import tqdm
import time
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd

In [185]:
NUM_EPOCHS = 50
BATCH_SIZE = 64
HIDDEN_SIZE = 512
MODEL_VERSION = 2
LEARNING_RATE = 0.01
CLASS_0_PERCENT = 0.1

In [186]:
cuda = torch.cuda.is_available()
num_workers = 8 if cuda else 0
DEVICE = "cuda" if cuda else "cpu"
print("Cuda = "+str(cuda)+" with num_workers = "+str(num_workers))

Cuda = True with num_workers = 8


In [187]:
DEBUG = False

In [188]:
class SpecializedDataset(Dataset):
    
    def __init__(self, datapath, mode, task_name, phonemes_class_0, phonemes_class_1, phonemes_class_2):
      """

      phonemes_class_0: list of phoneme names for class 0
      phonemes_class_1: list of phoneme names for class 1
      phonemes_class_2: list of phoneme names for class 1
      """
      complete_features = np.zeros((1, 40))  # eliminate this row
      complete_labels = np.zeros((1))  # eliminate this row

      # go through all files in datapath, check phoneme if class=0 or class=1
      # assign that label and discard silence frames
      class_0_phonemes_found = []
      class_1_phonemes_found = []
      class_2_phonemes_found = []
      silence_added = False
      with os.scandir(datapath) as entries:
        for entry in entries:
          if entry.is_file():
            if "features" in entry.name and mode in entry.name:
              phoneme_tag = entry.name.split("_")[0]

              # find phoneme in list and assign label
              phoneme_class = None
              if phoneme_tag in phonemes_class_0:
                phoneme_class = 0
                class_0_phonemes_found.append(phoneme_tag)
              if phoneme_tag in phonemes_class_1:
                phoneme_class = 1
                class_1_phonemes_found.append(phoneme_tag)
              if phoneme_tag in phonemes_class_2:
                phoneme_class = 2
                class_2_phonemes_found.append(phoneme_tag)

              if phoneme_class is None:
                print(f"phoneme '{phoneme_tag}' not found on any list; skip") if DEBUG else None
                continue

              print(f"phoneme '{phoneme_tag}' is class: {phoneme_class}") if DEBUG else None

              features_filepath = entry.path
              labels_filepath = f"{datapath}/{phoneme_tag}_{mode}_labels.npy"

              phoneme_features = np.load(features_filepath, allow_pickle=True)
              phoneme_labels = np.load(labels_filepath, allow_pickle=True)
              print(f"{phoneme_tag} total features: {phoneme_features.shape}") if DEBUG else None
              print(f"{phoneme_tag} total labels: {phoneme_labels.shape}") if DEBUG else None

              # find frames where label != 0 (non-silence)
              non_zero_indexes = phoneme_labels.nonzero()
              # NOTE: SIL is hardcoded as class_0 for all specialized task classifiers
              phoneme_labels[non_zero_indexes] = phoneme_class  # label=class

              if silence_added:
                phoneme_features = phoneme_features[non_zero_indexes]
                phoneme_labels = phoneme_labels[non_zero_indexes]
              else:
                silence_added = True
              
              end_index = int(CLASS_0_PERCENT * len(phoneme_features))
              
              # stack to phoneme features
              complete_features = np.concatenate((complete_features, phoneme_features[:end_index]))
              complete_labels = np.concatenate((complete_labels, phoneme_labels[:end_index]))
      
      self.X = complete_features[1:]
      self.Y = complete_labels[1:]
      print(f"[task={task_name}] {self.X.shape} features")
      print(f"[task={task_name}] {self.Y.shape} labels")

      if sorted(class_1_phonemes_found) != sorted(phonemes_class_1):
        raise Exception(f"class 1 phonemes found ({sorted(class_1_phonemes_found)}) != expected phonemes ({sorted(phonemes_class_1)})")
 
      if sorted(class_2_phonemes_found) != sorted(phonemes_class_2):
        raise Exception(f"class 2 phonemes found ({sorted(class_2_phonemes_found)}) != expected phonemes ({sorted(phonemes_class_2)})")
 

    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, index):
        x = torch.Tensor(self.X[index]).float()
        y = torch.as_tensor(self.Y[index]).long()
    
        return x,y

In [189]:
def make_dataloader(dataset, train, batch_size):
  if train:
    shuffle = True
    drop_last = True
  else:
    shuffle = False
    drop_last = False
    
  loader = DataLoader(dataset=dataset, batch_size=batch_size,
                      drop_last=drop_last, shuffle=shuffle,
                      pin_memory=True, num_workers=8)
  
  return loader

In [190]:
class SpecializedShallowDetector(nn.Module):
  
  def __init__(self, hidden_size, activation):
    super(SpecializedShallowDetector, self).__init__()
    
    self.linear_layer = nn.Linear(in_features=40, out_features=hidden_size)
    self.bn_layer = nn.BatchNorm1d(num_features=hidden_size)
    self.activation = activation
    self.output_layer = nn.Linear(in_features=hidden_size, out_features=3)
    seq_params = [
      self.linear_layer,
      self.bn_layer,
      self.activation,
      self.output_layer
    ]

    self.network = nn.Sequential(*seq_params)
    
  def forward(self, x):
    return self.network(x)

In [191]:
class SpecializedDetector():

  def __init__(self, task_name, phonemes_class_0, phonemes_class_1, phonemes_class_2):
    self.task_name = task_name

    train_data = SpecializedDataset(datapath=datapath, mode="train", 
                                    task_name=task_name, 
                                    phonemes_class_0=phonemes_class_0, 
                                    phonemes_class_1=phonemes_class_1,
                                    phonemes_class_2=phonemes_class_2)
    self.train_loader = make_dataloader(dataset=train_data, train=True, batch_size=BATCH_SIZE)

    dev_data = SpecializedDataset(datapath=datapath, mode="dev", 
                                  task_name=task_name, 
                                  phonemes_class_0=phonemes_class_0, 
                                  phonemes_class_1=phonemes_class_1,
                                  phonemes_class_2=phonemes_class_2)
    self.dev_loader = make_dataloader(dataset=dev_data, train=False, batch_size=BATCH_SIZE)

    self.model = SpecializedShallowDetector(hidden_size=HIDDEN_SIZE, 
                                            activation=nn.LeakyReLU()).to(DEVICE)
    self.criterion = nn.CrossEntropyLoss()
    self.optimizer = torch.optim.SGD(self.model.parameters(), lr=LEARNING_RATE, momentum=0.9)
    self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min')

    self.softmax = nn.Softmax(dim=1)

    self.train_loss_per_epoch = []
    self.train_acc_per_epoch = []
    self.dev_loss_per_epoch = []
    self.dev_acc_per_epoch = []
  
  def save_model(self, epoch):
    model_epoch_path = "{}/models/model_{}_{}_{}".format(output_path, self.task_name, 
                                                         MODEL_VERSION, epoch)
    torch.save({
        'model_state_dict': self.model.state_dict(),
        'optimizer_state_dict': self.optimizer.state_dict(),
        'scheduler_state_dict': self.scheduler.state_dict(),
    }, model_epoch_path)
    # print('saved model: {}'.format(model_epoch_path))

  def train(self, epochs):
    # Run training and track with wandb
    total_batches = len(self.train_loader) * epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0

    for epoch in tqdm(range(epochs)):
        train_loss = 0.0
        start_time = time.time()
        total_predictions = 0
        correct_predictions = 0

        true_labels = []
        predictions = []
        for _, (features, targets) in enumerate(self.train_loader):
            features, targets = features.to(DEVICE), targets.to(DEVICE)

            batch_loss, outputs = self.train_batch(features, targets)
            train_loss += batch_loss

            example_ct += len(features)
            batch_ct += 1
            outputs = outputs.to(DEVICE)

            # check number of correct predictions
            output_classes = torch.argmax(outputs.log_softmax(1), dim=1)  # convert to class labels
            total_predictions += len(output_classes)
            correct_predictions += torch.sum(targets == output_classes)

            true_labels += targets.detach().cpu().tolist()
            predictions += output_classes.detach().cpu().tolist()

        end_time = time.time()

        train_loss /= example_ct
        print(f"training loss: {train_loss}; time: {end_time - start_time}s")
        
        if (epoch + 1) % 10 == 0 or epoch == (epochs - 1):
          report = classification_report(true_labels, predictions, output_dict=True)
          df = pd.DataFrame(report).transpose()
          df.to_csv(f"{output_path}/reports/train_{self.task_name}_{MODEL_VERSION}_{epoch + 1}.csv", index=False)

        print(classification_report(true_labels, predictions))
        print(confusion_matrix(true_labels, predictions))

        self.train_loss_per_epoch.append(train_loss)
        #self.train_acc_per_epoch.append(train_acc)

        # evaluate model with validation data
        dev_loss = self.evaluate_model(epoch)
        
        self.dev_loss_per_epoch.append(dev_loss)
        #self.dev_acc_per_epoch.append(dev_acc)

        # Step with the scheduler
        self.scheduler.step(dev_loss)
      
    # epoch completed, save model
    self.save_model(epoch)

  def train_batch(self, features, targets):
    self.optimizer.zero_grad()

    # Forward pass ➡
    outputs = self.model(features)
    loss = self.criterion(outputs, targets)  # compare with target outputs
    # Backward pass ⬅
    loss.backward()
    # Step with optimizer
    self.optimizer.step()

    return loss.item(), outputs

  def evaluate_model(self, epoch):

    with torch.no_grad():
      self.model.eval()

      running_loss = 0.0
      total_predictions = 0.0
      correct_predictions = 0.0

      true_labels = []
      predictions = []

      example_ct = 0
      start_time = time.time()
      for batch_idx, (features, targets) in enumerate(self.dev_loader):
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)

        example_ct += len(features)

        outputs = self.model(features)
        outputs = outputs.to(DEVICE)

        # check number of correct predictions
        output_classes = torch.argmax(outputs.log_softmax(1), dim=1)  # convert to class labels
        total_predictions += len(output_classes)
        correct_predictions += torch.sum(targets == output_classes)

        loss = self.criterion(outputs, targets).detach()
        running_loss += loss.item()

        true_labels += targets.detach().cpu().tolist()
        predictions += output_classes.detach().cpu().tolist()
      
      end_time = time.time()

      running_loss /= example_ct
      print(f"testing loss: {running_loss}; time: {end_time - start_time}s")

      if (epoch + 1) % 10 == 0:
        report = classification_report(true_labels, predictions, output_dict=True)
        df = pd.DataFrame(report).transpose()
        df.to_csv(f"{output_path}/reports/reports_dev_{self.task_name}_{MODEL_VERSION}_{epoch + 1}.csv", index=False)
  
      print(classification_report(true_labels, predictions))
      print(confusion_matrix(true_labels, predictions))
      return running_loss

## 2. Training shallow detectors (per phoneme + one for silence)

In [192]:
%cd /content/gdrive/MyDrive/Spring_2021/11785_Intro_to_Deep_Learning/dl-group-project-code/utilities

/content/gdrive/MyDrive/Spring_2021/11785_Intro_to_Deep_Learning/dl-group-project-code/utilities


In [193]:
from utilities import SPECIALIZED_TASKS

In [194]:
%cd /

/


In [195]:
print(SPECIALIZED_TASKS)

{'1_vowel_vs_consonant': {0: ['SIL', 'AY', 'B', 'D', 'DH', 'G', 'II', 'J', 'K', 'LL', 'OH', 'OW', 'OY', 'P', 'T', 'TH', 'CH'], 1: ['EE', 'IH', 'EH', 'AE', 'UH', 'ER', 'AH', 'AW', 'OO', 'UE'], 2: ['FF', 'HH', 'MM', 'NN', 'NG', 'RR', 'SS', 'SH', 'VV', 'WW', 'YY', 'ZZ']}, '3_highvowel_vs_lowvowel': {0: ['SIL', 'AY', 'B', 'EH', 'D', 'DH', 'FF', 'G', 'HH', 'II', 'J', 'K', 'LL', 'MM', 'NN', 'OH', 'OW', 'OY', 'P', 'RR', 'SH', 'SS', 'T', 'TH', 'UH', 'VV', 'WW', 'YY', 'ZZ', 'CH', 'ER', 'NG'], 1: ['EE', 'IH', 'UE', 'OO'], 2: ['AE', 'AH', 'AW']}, '4_voiced_vs_unvoiced_fricatives': {0: ['SIL', 'AE', 'AH', 'AW', 'AY', 'B', 'EH', 'D', 'EE', 'G', 'HH', 'IH', 'II', 'J', 'K', 'LL', 'MM', 'NN', 'OH', 'OO', 'OW', 'OY', 'P', 'RR', 'T', 'UE', 'UH', 'WW', 'YY', 'CH', 'ER', 'NG'], 1: ['DH', 'VV', 'ZZ'], 2: ['FF', 'SS', 'SH', 'TH']}, '5_ss_vs_zz': {0: ['SIL', 'AE', 'AH', 'AW', 'AY', 'B', 'EH', 'D', 'DH', 'EE', 'FF', 'G', 'HH', 'IH', 'II', 'J', 'K', 'LL', 'MM', 'NN', 'OH', 'OO', 'OW', 'OY', 'P', 'RR', 'SH', 'T

In [196]:
not_trained = []
for task_name, classes_dict in SPECIALIZED_TASKS.items():
  detector = SpecializedDetector(task_name, classes_dict[0], classes_dict[1], classes_dict[2])
  detector.train(epochs=NUM_EPOCHS)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))







  0%|          | 0/50 [00:00<?, ?it/s]

[task=1_vowel_vs_consonant] (2192, 40) features
[task=1_vowel_vs_consonant] (2192,) labels
[task=1_vowel_vs_consonant] (522, 40) features
[task=1_vowel_vs_consonant] (522,) labels
training loss: 0.014284111778525746; time: 0.5763533115386963s
              precision    recall  f1-score   support

           0       0.54      0.63      0.58       898
           1       0.55      0.41      0.47       517
           2       0.60      0.59      0.59       761

    accuracy                           0.56      2176
   macro avg       0.56      0.54      0.55      2176
weighted avg       0.56      0.56      0.56      2176

[[566 116 216]
 [221 211  85]
 [258  55 448]]









  2%|▏         | 1/50 [00:01<00:53,  1.09s/it]

testing loss: 0.015704819297425135; time: 0.4913947582244873s
              precision    recall  f1-score   support

           0       0.46      0.43      0.45       184
           1       0.51      0.55      0.53       150
           2       0.61      0.61      0.61       188

    accuracy                           0.53       522
   macro avg       0.53      0.53      0.53       522
weighted avg       0.53      0.53      0.53       522

[[ 79  49  56]
 [ 49  83  18]
 [ 43  30 115]]
training loss: 0.006579210981726646; time: 0.5634303092956543s
              precision    recall  f1-score   support

           0       0.61      0.60      0.60       900
           1       0.60      0.58      0.59       515
           2       0.64      0.66      0.65       761

    accuracy                           0.62      2176
   macro avg       0.62      0.61      0.61      2176
weighted avg       0.62      0.62      0.62      2176

[[536 153 211]
 [139 301  75]
 [210  46 505]]









  4%|▍         | 2/50 [00:02<00:52,  1.09s/it]

...

testing loss: 0.0007032543515351198; time: 0.7077479362487793s
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       483
           1       1.00      0.89      0.94        19
           2       0.87      1.00      0.93        20

    accuracy                           0.99       522
   macro avg       0.96      0.96      0.96       522
weighted avg       0.99      0.99      0.99       522

[[480   0   3]
 [  2  17   0]
 [  0   0  20]]
training loss: 1.845237778759386e-05; time: 0.7850027084350586s
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2024
           1       0.97      0.76      0.85        79
           2       1.00      0.84      0.91        73

    accuracy                           0.98      2176
   macro avg       0.98      0.86      0.92      2176
weighted avg       0.98      0.98      0.98      2176

[[2022    2    0]
 [  19   60    0]
 [  12    0   61]]









 98%|█████████▊| 49/50 [01:11<00:01,  1.47s/it]

testing loss: 0.0006999679568127312; time: 0.6636207103729248s
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       483
           1       1.00      0.89      0.94        19
           2       0.87      1.00      0.93        20

    accuracy                           0.99       522
   macro avg       0.96      0.96      0.96       522
weighted avg       0.99      0.99      0.99       522

[[480   0   3]
 [  2  17   0]
 [  0   0  20]]
training loss: 1.7338955119211117e-05; time: 0.7717328071594238s
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2028
           1       0.97      0.77      0.86        77
           2       1.00      0.85      0.92        71

    accuracy                           0.99      2176
   macro avg       0.98      0.87      0.92      2176
weighted avg       0.99      0.99      0.99      2176

[[2026    2    0]
 [  18   59    0]
 [  11    0   60]]









100%|██████████| 50/50 [01:12<00:00,  1.45s/it]

testing loss: 0.0007033499753480899; time: 0.6745748519897461s
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       483
           1       1.00      0.89      0.94        19
           2       0.87      1.00      0.93        20

    accuracy                           0.99       522
   macro avg       0.96      0.96      0.96       522
weighted avg       0.99      0.99      0.99       522

[[480   0   3]
 [  2  17   0]
 [  0   0  20]]


In [ ]:
for task_name, error in not_trained:
  print(f"'{task_name}' not trained")
  print(f"error: {error}")
  print("")